# Classification using BFGS -- Pytorch version

This notebook details the implementation of a generic ridge-regularized classification solved by direct gradient-based optimization (here quasi-newton). 
It is implemented in the kernel space, i.e. representing the weights over the space of points.

In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device_cpu = torch.device("cpu")
device_cpu = device
print( device )

mytype = torch.float16 # to save memory (only on GPU)
mytype = torch.float32

cpu


# Data

In [2]:
%load_ext autoreload
%autoreload 2

import utils
from utils import load_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df_DB = load_data()
print(df_DB.shape)
df_DB.head()

(13717, 7)


,uniprot,DBid,smiles,ind2mol,fasta,ind2prot,inter
0,A0A024R8I1,DB00381,CCOC(=O)C1=C(COCCN)NC(C)=C(C1C1=CC=CC=C1Cl)C(=...,213,MVRFGDELGGRYGGPGGGERARGGGAGGAGGPGPGGLQPGQRVLYK...,0,1
1,A0A024R8I1,DB00996,NCC1(CC(O)=O)CCCCC1,686,MVRFGDELGGRYGGPGGGERARGGGAGGAGGPGPGGLQPGQRVLYK...,0,1
2,A1L3X4,DB12965,[Ag],4672,MDLSCSCATGGSCTCASSCKCKEYKCTSCKKNCCSCCPMGCAKCAQGCT,1,1
3,A5X5Y0,DB00715,FC1=CC=C(C=C1)[C@@H]1CCNC[C@H]1COC1=CC2=C(OCO2...,462,MEGSWFHRKRFSFYLLLGFLLQGRGVTFTINCSGFGQHGADPTALN...,2,1
4,A5X5Y0,DB09304,CN1CCC2=C(C1)C1=CC=CC=C1CC1=CC=CC=C21,4467,MEGSWFHRKRFSFYLLLGFLLQGRGVTFTINCSGFGQHGADPTALN...,2,1


In [4]:
df_DB[df_DB["smiles"] =="NC1=C(C2=C(N)N=C(N)N=C2C=C1)[Cl](=O)=O"]
# drop the 2 molecules with the same smiles
df_DB = df_DB.drop(12222)
df_DB[df_DB["smiles"] =="NC1=C(C2=C(N)N=C(N)N=C2C=C1)[Cl](=O)=O"]


,uniprot,DBid,smiles,ind2mol,fasta,ind2prot,inter


In [5]:
df_DB.shape

(13716, 7)

In [25]:
df_DB.to_csv('data/drugbank.csv', index=False)

## Kprot

In [26]:
import pickle
with open('data/drugbank_K_prot.data', 'rb') as f:
        K_prot = pickle.load(f)

In [27]:
K_prot.shape

(2513, 2513)

## liste des 4814 smiles

In [28]:
# same in zip format
import pandas as pd
import zipfile
zf = zipfile.ZipFile('data/drugbank.csv.zip') 
df = pd.read_csv(zf.open('drugbank.csv'),low_memory=False)
df_p = df[df['inter'] == True]
#list of smiles strings
smiles = df_p['smiles'].drop_duplicates().values
len(smiles)

4813

In [29]:
from rdkit import Chem
from rdkit.Chem import AllChem

import numpy as np

nM =  len(smiles)
MorganFP = np.zeros((nM,1024))
for i in range(nM):
    # Convert SMILES to RDKit molecule object
    mol = Chem.MolFromSmiles(smiles[i])    
    # Generate Morgan fingerprint of the molecule
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
    # Convert the fingerprint to a numpy array
    arr = np.zeros((1,))
    AllChem.DataStructs.ConvertToNumpyArray(fp, arr)
    MorganFP[i,:] = arr
MorganFP = MorganFP.astype(int)

[22:22:02] Unusual charge on atom 0 number of radical electrons set to zero


In [30]:
import Nystrom_method
from  Nystrom_method import nystroem,KronKernel
# random list of molecules 
kM = 4814 # number of molecule to compute nystrom
rM = 1000 # final dimension of features
I = np.random.permutation(nM)
I = I[:kM]

In [31]:
# compute Tanimoto kernel 
Km = ( MorganFP[I,:] @ MorganFP.T ) / ( 1024 - (1-MorganFP[I,:]) @ (1-MorganFP.T) )

In [32]:
Xm,Lambda,LambdaC = nystroem(np.concatenate((Km[:,I], Km), axis=1),rM)

## liste des indices protéines/molécules avec que des 1

In [34]:
# protein indices
J = df_p['ind2prot'].values
print(len(J))
# molecules indices
I = df_p['ind2mol'].values
print(len(I))

13716
13716


## train/test avec indices protéines/molécules et interactions balanced

en premier l'indice de la protéine, puis l'indice du ligand puis l'interaction

In [29]:
# change name of the column 'ind2prot' in 'indfasta' in df
df = df_DB.rename(columns={'ind2prot': 'indfasta', 'ind2mol': 'indsmiles', 'inter': 'score'})
df.head()

,uniprot,DBid,smiles,indsmiles,fasta,indfasta,score
0,A0A024R8I1,DB00381,CCOC(=O)C1=C(COCCN)NC(C)=C(C1C1=CC=CC=C1Cl)C(=...,213,MVRFGDELGGRYGGPGGGERARGGGAGGAGGPGPGGLQPGQRVLYK...,0,1
1,A0A024R8I1,DB00996,NCC1(CC(O)=O)CCCCC1,686,MVRFGDELGGRYGGPGGGERARGGGAGGAGGPGPGGLQPGQRVLYK...,0,1
2,A1L3X4,DB12965,[Ag],4672,MDLSCSCATGGSCTCASSCKCKEYKCTSCKKNCCSCCPMGCAKCAQGCT,1,1
3,A5X5Y0,DB00715,FC1=CC=C(C=C1)[C@@H]1CCNC[C@H]1COC1=CC2=C(OCO2...,462,MEGSWFHRKRFSFYLLLGFLLQGRGVTFTINCSGFGQHGADPTALN...,2,1
4,A5X5Y0,DB09304,CN1CCC2=C(C1)C1=CC=CC=C1CC1=CC=CC=C21,4467,MEGSWFHRKRFSFYLLLGFLLQGRGVTFTINCSGFGQHGADPTALN...,2,1


In [30]:
from utils import make_train_test

all_train_interactions_arr, all_test_interactions_arr = make_train_test(df_name,5,1)

train (21944, 3)
test (5488, 3)
train (21946, 3)
test (5486, 3)


In [ ]:
test = all_test_interactions_arr[0]
train = all_train_interactions_arr[0]
c = 0
for elt in test[:,:]:
    if elt in train[:,:]:
        c+=1
        print(elt)
print(c)
print(len(test[:,0:2]))

In [11]:
# algo Matthieu corrected
intMat = df.pivot(index='indfasta', columns="indsmiles", values='score').to_numpy(dtype=np.float16)
intMat

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float16)

In [23]:
n_p,n_m = intMat.shape
Ip, Jm = np.where(intMat==1)
print(Ip,Jm,intMat[0,213])
nb_positive_inter = int(len(Ip))
Inp, Jnm = np.where(intMat==0)
Inkp, Jnkm = np.where(np.isnan(intMat))
print(Inkp,Jnkm,intMat[0,0])


[   0    0    1 ... 2512 2512 2512] [ 213  686 4671 ... 1563 1670 3947] 1.0
[   0    0    0 ... 2512 2512 2512] [   0    1    2 ... 4811 4812 4813] nan


In [24]:
from sklearn import model_selection
skf_positive = model_selection.KFold(shuffle=True, n_splits=5)
for train_index, test_index in skf_positive.split(range(nb_positive_inter)):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [    1     2     5 ... 13713 13714 13715] TEST: [    0     3     4 ... 13695 13696 13697]
TRAIN: [    0     1     2 ... 13713 13714 13715] TEST: [    6     9    11 ... 13672 13700 13704]
TRAIN: [    0     1     2 ... 13711 13712 13714] TEST: [    7    14    18 ... 13707 13713 13715]
TRAIN: [    0     1     2 ... 13710 13713 13715] TEST: [    5    21    31 ... 13711 13712 13714]
TRAIN: [    0     3     4 ... 13713 13714 13715] TEST: [    1     2     8 ... 13706 13708 13710]


In [28]:
Ip[train_index],Jm[train_index],intMat[2512,1563]

(array([   0,    2,    2, ..., 2512, 2512, 2512]),
 array([ 213,  462, 4466, ..., 1563, 1670, 3947]),
 1.0)

In [ ]:
def make_train_test(df,nb_folds):
  """
    make train and test sets
  """

  # algo Matthieu corrected
  intMat = df.pivot(index='indfasta', columns="indsmiles", values='score').to_numpy(dtype=np.float16)

  # Set the different folds
  skf_positive = model_selection.KFold(shuffle=True, n_splits=nb_folds)

  all_train_interactions_arr = []
  all_test_interactions_arr = []

  n_p,n_m = intMat.shape
  Ip, Jm = np.where(intMat==1)
  nb_positive_inter = int(len(Ip))
  Inp, Jnm = np.where(intMat==0)
  Inkp, Jnkm = np.where(np.isnan(intMat))

  for train_index, test_index in skf_positive.split(range(nb_positive_inter)):
      # 9' pour train
      train_index = np.random.choice(train_index, int(p*len(train_index)), replace=False)

      Mm, bin_edges = np.histogram(Ip[train_index], bins = range(n_p+1)) # np.array with  #interactions for each protein of the train at the beginning

      Mp, bin_edges = np.histogram(Jm[train_index], bins = range(n_m+1)) # np.array with  #interactions for each drugs at the beginning (how manu time it can be chosen)

      train = np.zeros([1,3], dtype=int)

      nb_prot = len(list(set(Ip[train_index]))) # number of different prot in train
      for i in range(nb_prot):

          j = np.argmax(Mm) # choose protein with the maximum of interactions in the train

          indice_P = Jm[train_index][np.where(Ip[train_index]==j)[0]]  #np.array with index of interactions + in train
          indice_N = [k for k in Jm[train_index] if intMat[j][k]==0]
          indice_NK = [k for k in Jm[train_index] if np.isnan(intMat[j][k])] #np.array  with index of interactions not known

          indice_freq_mol = np.where(Mp>1)[0]  #drug's index with more than 2 interactions +
          indice_poss_mol = np.where(Mp == 1)[0]  #drug's index with 1 interaction +

          indice_freq_one_prot = np.intersect1d(indice_N, indice_freq_mol)
          indice_poss_one_prot = np.intersect1d(indice_N, indice_poss_mol)

          nb_positive_interactions = len(indice_P)
          nb_frequent_hitters_negative_interactions = len(indice_freq_one_prot)

          indice_freq_one_prot = np.intersect1d(indice_N, indice_freq_mol)
          indice_poss_one_prot = np.intersect1d(indice_N, indice_poss_mol)
          indice_freq_one_prot_NK = np.intersect1d(indice_NK, indice_freq_mol)
          indice_poss_one_prot_NK = np.intersect1d(indice_NK, indice_poss_mol)

          if len(indice_P) <= len(indice_freq_one_prot):
              # we shoot at random nb_positive_interactions in drugs with a lot of interactions
              indice_N_one_prot = np.random.choice(indice_freq_one_prot,
                                                  len(indice_P), replace = False)
          elif len(indice_P) <= len(indice_freq_one_prot) + len(indice_poss_one_prot):
              # we shoot at random nb_positive_interactions in drugs with a lot of interactions
              nb_negative_interactions_remaining = len(indice_P) - len(indice_freq_one_prot)
              indice_N_one_prot_poss = np.random.choice(indice_poss_one_prot,
                                                      nb_negative_interactions_remaining, replace = False )
              indice_N_one_prot = np.concatenate((indice_freq_one_prot,
                                              indice_N_one_prot_poss))
          elif len(indice_P) <= len(indice_freq_one_prot) + len(indice_poss_one_prot) + len(indice_freq_one_prot_NK):
              # we shoot at random nb_positive_interactions in drugs with a lot of interactions
              nb_negative_interactions_remaining = len(indice_P) - len(indice_freq_one_prot) - len(indice_poss_one_prot)
              indice_N_one_prot_poss = np.random.choice(indice_freq_one_prot_NK,
                                                      nb_negative_interactions_remaining, replace = False )
              indice_N_one_prot = np.concatenate((indice_freq_one_prot,
                                              indice_poss_one_prot, indice_N_one_prot_poss))
          else:
              # we shoot at random nb_positive_interactions in drugs with a lot of interactions
              nb_negative_interactions_remaining = len(indice_P) - len(indice_freq_one_prot) - len(indice_poss_one_prot) - len(indice_freq_one_prot_NK)
              #print("nb_negative_interactions_remaining", nb_negative_interactions_remaining) # pas de solution...
              #print(indice_poss_one_prot_NK.shape)
              indice_N_one_prot_poss = np.random.choice(indice_poss_one_prot_NK,
                                                      nb_negative_interactions_remaining, replace = False )
              indice_N_one_prot = np.concatenate((indice_freq_one_prot,
                                              indice_poss_one_prot, indice_freq_one_prot_NK, indice_N_one_prot_poss))

          Mp[indice_N_one_prot.astype(int)]-=1

          # this protein has been processed
          Mm[j] = 0

          indice = np.r_[indice_P,indice_N_one_prot].astype(int)
          etiquette = [x if not np.isnan(x) else 0 for x in intMat[j][indice]]
          A = np.stack((indice, etiquette), axis=-1)
          B = np.c_[np.zeros(A.shape[0])+j,A].astype(int)
          train = np.concatenate((train,B))

      train = train[1:]
      all_train_interactions_arr.append(train)
      print("train", train.shape)


      # test
      #test_index =  np.random.choice(test_index, int(p*len(test_index)), replace=False)
      # interactions + in test
      indice_P_t = np.c_[Ip[test_index],Jm[test_index], np.ones(len(test_index))].astype(int)

      # interactions - in test
      a = np.r_[np.c_[Inp,Jnm]] # all the zeros in the matrix (and NK ?)
      a1 = set(map(tuple, a))
      b = train[:,:2]   # all the interactions in the train
      b1 = set(map(tuple, b))
      indice_N_t = np.array(list(a1 - b1))#[:indice_P_t.shape[0],:] # we keep the same number of interactions - than interactions + in test, choosing the 0 in the matrix
      #print(len(indice_N_t))

      # add interactions np.nan in test

      if len(indice_N_t) == 0:
          # initialization
          indice_N_t = np.array([-1, -1]).reshape(1,2)

      c = np.r_[np.c_[Inkp,Jnkm]] # all the np.nan in the matrix

      if len(indice_N_t) < indice_P_t.shape[0]:
          # we add some interactions - in test to have the same number of interactions + and - in test choose in the np.nan in the matrix
          k = 0
          while len(indice_N_t) < indice_P_t.shape[0]+1:
              i = np.random.randint(0, len(c))
              if tuple(c[i]) not in b1:
                  indice_N_t = np.concatenate((indice_N_t, c[i].reshape(1,2)))
                  k += 1

      # we drop the first row of indice_N_t if is [-1, -1]
      if indice_N_t[0,0] == -1:
          indice_N_t = indice_N_t[1:,:]

      indice_N_t = indice_N_t[:len(indice_P_t),:]

      # we add the column of 0 for the etiquette
      indice_N_t = np.c_[indice_N_t, np.zeros(len(indice_N_t))].astype(int)
      test = np.r_[indice_P_t,indice_N_t]

      all_test_interactions_arr.append(test)
      print("test", test.shape)

  print("Train/test datasets prepared.")
  return all_train_interactions_arr, all_test_interactions_arr